In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from pymorphy2 import MorphAnalyzer
import re
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, ImageColorGenerator

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
ds = pd.read_csv('reviews.csv', sep='\t') #Загрузили датасет

In [3]:
ds.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [4]:
ds.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     90000 non-null  object
 1   sentiment  90000 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [5]:
ds.describe()

,review,sentiment
count,90000,90000
unique,87321,3
top,Товар не пришёл,positive
freq,58,30000


In [6]:
ds.duplicated().sum() #повторяются в работе столько-то рецензий

2250

In [7]:
#Удалим дубликаты и неуникальные значения. Дублирующиеся данные могут не вызывать ошибок, но отнимать много времени для получения результата.
df = ds.drop_duplicates(subset=['review'])

In [8]:
df.describe() #freq - частота наиболее распространенного значения

,review,sentiment
count,87321,87321
unique,87321,3
top,Ждала посылку 2 месяца в итоге получила с горе...,positive
freq,1,29291


In [9]:
df.sentiment.value_counts() #сколько каких в таблице. Было везде по 30к

positive    29291
negative    29218
neautral    28812
Name: sentiment, dtype: int64

In [10]:
df.info() #Нет пустых значений

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87321 entries, 0 to 89999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     87321 non-null  object
 1   sentiment  87321 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [11]:
# разобьем общий датасет и сделаем в будущем классификацию на позитивный и негативный
data_pos=df[df['sentiment'] == 'positive'][['review', 'sentiment']]
data_neg = df[df['sentiment'] == 'negative'][['review', 'sentiment']]

# Удаление стоп-слов, токенизация, лемматизация

In [12]:
stop_words = stopwords.words("russian")
morph = MorphAnalyzer()

In [13]:
def get_clean_tokens(sentence):
    #преобразование в одинаковый регистр
    tokens = re.findall("\w+", sentence.lower()) #\w+ - один или больше alphanumeric-символов, то же, что и [a-zA-Z0-9]+
    tokens_no_stops = [word for word in tokens if (word not in punctuation) and (word not in stop_words)]
    tokens_no_singles = [token for token in tokens_no_stops if len(token) > 4]
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens_no_singles]
    return lemmatized_tokens

In [14]:
#Разбиение позитивных токенов
pos_tok = [get_clean_tokens(sentence) for sentence in data_pos['review']]
pos_tok_join = [' '.join(sent) for sent in pos_tok]

In [15]:
#Негативных
neg_tok = [get_clean_tokens(sentence) for sentence in data_neg['review']]
neg_tok_join = [' '.join(sent) for sent in neg_tok]

In [172]:
#Объединение
data_full = pd.concat([data_pos, data_neg])

# Векторизация текста Word2Vec

In [173]:
data_full.head(10)

,review,sentiment
60000,Все совпадает! К покупке рекомендую!,positive
60001,по размеру) качественные),positive
60002,"Прекрасная, качественая рубашка! Искала просту...",positive
60003,"Пришло через 2 месяца. Очень долго, но пуловер...",positive
60004,Заказ дошел быстро. Соответствует описанию. Кр...,positive
60005,"Приятная к телу, L на 42 свободно, так как х...",positive
60006,"платье хорошего качества, нитки нигде не торча...",positive
60007,"просто огонь!!! очень крутая шапка, толстая и ...",positive
60008,"платье оставило двоякое впечатление,само оно с...",positive
60009,"Кофта не подошла, очень короткая и в руках и д...",positive


In [18]:
data_full['Preprocessed_texts'] = data_full.apply(lambda row: get_clean_tokens(row['review']), axis=1)

In [19]:
data_full.head(10)

,review,sentiment,Preprocessed_texts
60000,Все совпадает! К покупке рекомендую!,positive,"[совпадать, покупка, рекомендовать]"
60001,по размеру) качественные),positive,"[размер, качественный]"
60002,"Прекрасная, качественая рубашка! Искала просту...",positive,"[прекрасный, качественай, рубашка, искать, про..."
60003,"Пришло через 2 месяца. Очень долго, но пуловер...",positive,"[прийти, месяц, очень, долго, пуловер, хороший..."
60004,Заказ дошел быстро. Соответствует описанию. Кр...,positive,"[заказ, дойти, быстро, соответствовать, описан..."
60005,"Приятная к телу, L на 42 свободно, так как х...",positive,"[приятный, свободно, хотеть]"
60006,"платье хорошего качества, нитки нигде не торча...",positive,"[платье, хороший, качество, нитка, нигде, торч..."
60007,"просто огонь!!! очень крутая шапка, толстая и ...",positive,"[просто, огонь, очень, крутой, шапка, толстой,..."
60008,"платье оставило двоякое впечатление,само оно с...",positive,"[платье, оставить, двоякий, впечатление, симпа..."
60009,"Кофта не подошла, очень короткая и в руках и д...",positive,"[кофта, подойти, очень, короткий, рука, длинны..."


In [20]:
from gensim.models import Word2Vec

In [21]:
model = Word2Vec(sentences=data_full['Preprocessed_texts'], 
                               min_count=5, 
                               vector_size=100)

In [22]:
model.wv['платье']

array([ 0.10646771, -0.21461429, -1.1646378 , -0.8988447 ,  0.06372738,
        0.63265973, -0.61981136,  0.5556398 ,  0.520582  , -0.16672827,
       -1.0388619 , -0.34440282, -0.7948092 ,  0.56162405, -0.39096493,
        0.37631896,  0.31832027,  0.7178659 ,  0.41525674, -0.2389673 ,
       -0.20334497,  0.4970843 , -0.81829214,  0.90541226, -0.76176226,
        0.69675386, -0.23174393, -0.19571745, -0.3160934 ,  0.12680945,
       -0.29070958,  0.61647964,  0.35552654, -0.36672172, -1.2252364 ,
        0.13014157,  0.49328014,  0.48192468, -0.8562838 ,  0.15190671,
        0.81927687, -0.26153144,  0.04490416, -0.5579957 ,  0.08361428,
        0.6516333 , -0.37429297, -0.10132997,  0.5924949 ,  0.02552158,
        0.5353518 , -0.672938  ,  0.6774272 , -0.42333   , -0.46273762,
       -0.33624628,  0.969439  , -0.1279555 , -0.3281217 ,  0.88015765,
        0.8679009 ,  0.05117419,  0.2651446 , -0.21226658,  0.2242977 ,
        0.8402036 ,  0.05043481, -0.9195437 , -0.49842545,  0.80

In [23]:
model.wv.most_similar('хороший') #"плохой", потому что грамматически неправильно люди пишут "не плохой"

[('отличный', 0.9047948718070984),
 ('неплохой', 0.8533983826637268),
 ('высота', 0.8014172911643982),
 ('прекрасный', 0.7884454727172852),
 ('плохой', 0.7717111110687256),
 ('достойный', 0.7683626413345337),
 ('классный', 0.7637544870376587),
 ('нормальный', 0.7614010572433472),
 ('супер', 0.7331649661064148),
 ('замечательный', 0.7320221066474915)]

In [24]:
model.wv.most_similar('плохой') 
#Слово хороший там потому что в отзывах выделяют еще и положительные стороны товара, помимо отрицательного. 
#Например, "К сожалению, заказ не отслеживается. Пропал... очень обидно... обратная связь с продавцом хорошая 	negative"

[('ужасный', 0.7983976006507874),
 ('отвратительный', 0.7967720627784729),
 ('хороший', 0.7717110514640808),
 ('низкий', 0.7644712924957275),
 ('неплохой', 0.7312021255493164),
 ('достойный', 0.6958544254302979),
 ('среднее', 0.6947188973426819),
 ('нормальный', 0.6576545238494873),
 ('троечка', 0.645455002784729),
 ('приличный', 0.6194963455200195)]

In [31]:
def sent_vec(sent):
    vector_size = model.wv.vector_size
    wv_res = np.zeros(vector_size)
    ctr = 1
    for w in sent:
        if w in model.wv:
            ctr += 1
            wv_res += model.wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [32]:
data_full['vec'] = data_full['Preprocessed_texts'].apply(sent_vec)

In [33]:
data_full.head(10)

,review,sentiment,Preprocessed_texts,vec
60000,Все совпадает! К покупке рекомендую!,positive,"[совпадать, покупка, рекомендовать]","[0.03651978820562363, 0.09263061359524727, -0...."
60001,по размеру) качественные),positive,"[размер, качественный]","[-0.4697702080011368, 0.29086120923360187, 0.4..."
60002,"Прекрасная, качественая рубашка! Искала просту...",positive,"[прекрасный, качественай, рубашка, искать, про...","[-0.40640390730307746, 0.4971312520792708, -0...."
60003,"Пришло через 2 месяца. Очень долго, но пуловер...",positive,"[прийти, месяц, очень, долго, пуловер, хороший...","[-0.648510068655014, 0.2048019063141611, 0.031..."
60004,Заказ дошел быстро. Соответствует описанию. Кр...,positive,"[заказ, дойти, быстро, соответствовать, описан...","[-0.16213513978502966, -0.0064870691434903574,..."
60005,"Приятная к телу, L на 42 свободно, так как х...",positive,"[приятный, свободно, хотеть]","[-0.6595089286565781, 0.26119846291840076, -0...."
60006,"платье хорошего качества, нитки нигде не торча...",positive,"[платье, хороший, качество, нитка, нигде, торч...","[-0.3690585870295763, 0.2577522377949208, -0.2..."
60007,"просто огонь!!! очень крутая шапка, толстая и ...",positive,"[просто, огонь, очень, крутой, шапка, толстой,...","[-0.5783362186585481, 0.3598443610736957, 0.24..."
60008,"платье оставило двоякое впечатление,само оно с...",positive,"[платье, оставить, двоякий, впечатление, симпа...","[-0.5789234221336388, 0.2829572789016224, 0.25..."
60009,"Кофта не подошла, очень короткая и в руках и д...",positive,"[кофта, подойти, очень, короткий, рука, длинны...","[-0.32096708193421364, 0.47224746613452834, -0..."


# Подготовка выборки

In [174]:
mapping = {'negative': 0, 'positive': 1}

In [176]:
data_full.replace({'sentiment': mapping}, inplace=True)

In [177]:
data_full.head(30000)

,review,sentiment
60000,Все совпадает! К покупке рекомендую!,1
60001,по размеру) качественные),1
60002,"Прекрасная, качественая рубашка! Искала просту...",1
60003,"Пришло через 2 месяца. Очень долго, но пуловер...",1
60004,Заказ дошел быстро. Соответствует описанию. Кр...,1
...,...,...
704,"кофта в разрезе расходится по шву, очень обидно",0
705,Тысячи рублей толстовка не стоит. Размер заказ...,0
706,Заказ не получил. Продавец вернул только 2 дол...,0
707,"На 2 размера меньше пришел , плохо прошит , по...",0


# Деление на тренировочную и тестовую выборку V2

In [84]:
X = data_full['vec'].to_list()
y = data_full['sentiment'].to_list()

In [85]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

# Обучение

In [121]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [122]:
classifier.fit(X_train,y_train)

C:\Users\dom\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [123]:
from sklearn import metrics
predicted = classifier.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted, y_test)
f1_score = metrics.f1_score(predicted, y_test)

In [124]:
print(str('{:.1%}'.format(accuracy_score)))
print(str('{:.1%}'.format(f1_score)))

88.0%
87.8%


# Прогнозирование

In [166]:
text = """Хочу поблагодарить Наталью Юрьевну за возможность заказа с 30% скидкой! Наконец я смогла заказать отличную блузу!Спасибо!Мечты сбываются!"""

In [167]:
pred = get_clean_tokens(text)

In [168]:
predv = sent_vec(pred)

In [169]:
result = classifier.predict(predv.reshape(1, -1))

In [170]:
result

array([1])